In [1]:
df = spark.sql("select * from ab_nyc_2019_csv")

In [2]:
df.count()

Out[4]: 49078

In [3]:
df.describe()

Out[5]: DataFrame[summary: string, id: string, name: string, host_id: string, host_name: string, neighbourhood_group: string, neighbourhood: string, latitude: string, longitude: string, room_type: string, price: string, minimum_nights: string, number_of_reviews: string, last_review: string, reviews_per_month: string, calculated_host_listings_count: string, availability_365: string]

In [4]:
df.printSchema()

root
-- id: integer (nullable = true)
-- name: string (nullable = true)
-- host_id: integer (nullable = true)
-- host_name: string (nullable = true)
-- neighbourhood_group: string (nullable = true)
-- neighbourhood: string (nullable = true)
-- latitude: float (nullable = true)
-- longitude: float (nullable = true)
-- room_type: string (nullable = true)
-- price: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- last_review: string (nullable = true)
-- reviews_per_month: float (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- availability_365: integer (nullable = true)

In [5]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
183| 199| 351| 372| 351| 351| 351| 351| 351| 351| 351| 351| 10377| 10377| 351| 351|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+

In [6]:
df=df.dropna(subset=['neighbourhood_group','neighbourhood','latitude','longitude','room_type','price','minimum_nights','availability_365','calculated_host_listings_count'])

In [7]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 0| 16| 0| 21| 0| 0| 0| 0| 0| 0| 0| 0| 10026| 10026| 0| 0|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+

In [8]:
# we dont really need last review date , name and host_name for our analysis part since this doesnt affect our price in logical way in real world.
df=df.drop('last_review','name','host_name')

In [9]:
df.printSchema()

root
-- id: integer (nullable = true)
-- host_id: integer (nullable = true)
-- neighbourhood_group: string (nullable = true)
-- neighbourhood: string (nullable = true)
-- latitude: float (nullable = true)
-- longitude: float (nullable = true)
-- room_type: string (nullable = true)
-- price: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- reviews_per_month: float (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- availability_365: integer (nullable = true)

In [10]:
df.count()

Out[13]: 48736

In [11]:
# reviews per month will be zero if the number of reviews are zero ( we can clearly see here the number of reviews and reviews_per_month are of same count when reviews_per_month is zero)
df.select('number_of_reviews','reviews_per_month').filter("reviews_per_month is null").count()

Out[14]: 10029

In [12]:
# replacing the null values of reviews_per_month to zero.
df=df.fillna({ 'reviews_per_month':0 } )

In [13]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+

In [14]:
df.count()

Out[17]: 48736

In [15]:
df.show()

+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group| neighbourhood|latitude|longitude| room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
2539| 2787| Brooklyn| Kensington|40.64749|-73.97237| Private room| 149| 1| 9| 0.21| 6| 365|
2595| 2845| Manhattan| Midtown|40.75362|-73.98377|Entire home/apt| 225| 1| 45| 0.38| 2| 355|
3647| 4632| Manhattan| Harlem|40.80902| -73.9419| Private room| 150| 3| 0| 0.0| 1| 365|
3831| 4869| Brooklyn| Clinton Hill|40.68514|-73.95976|Entire home/apt| 89| 1| 270| 4.64| 1| 194|
5022| 7192| Manhattan| East Harlem|40.79851|-73.94399|Entire home/apt| 80| 10| 9| 0.1| 1| 0|
5099| 7322| Manhattan| Murray Hill|40.74767| -73.975|Entire home/apt| 200| 3| 74| 0.59| 1| 129|
5121| 7356| Brooklyn|Bedford-Stuyvesant|40.68688|-73.95596| Private room| 60| 45| 49| 0.4| 1| 0|
5178| 8967| Manhattan| Hell's Kitchen|40.76489|-73.98493| Private room| 79| 2| 430| 3.47| 1| 220|
5203| 7490| Manhattan| Upper West Side|40.80178|-73.96723| Private room| 79| 2| 118| 0.99| 1| 0|
5238| 7549| Manhattan| Chinatown|40.71344|-73.99037|Entire home/apt| 150| 1| 160| 1.33| 4| 188|
5295| 7702| Manhattan| Upper West Side|40.80316|-73.96545|Entire home/apt| 135| 5| 53| 0.43| 1| 6|
5441| 7989| Manhattan| Hell's Kitchen|40.76076|-73.98867| Private room| 85| 2| 188| 1.5| 1| 39|
5803| 9744| Brooklyn| South Slope|40.66829|-73.98779| Private room| 89| 4| 167| 1.34| 3| 314|
6021| 11528| Manhattan| Upper West Side|40.79826|-73.96113| Private room| 85| 2| 113| 0.91| 1| 333|
6090| 11975| Manhattan| West Village| 40.7353|-74.00525|Entire home/apt| 120| 90| 27| 0.22| 1| 0|
6848| 15991| Brooklyn| Williamsburg|40.70837|-73.95352|Entire home/apt| 140| 2| 148| 1.2| 1| 46|
7097| 17571| Brooklyn| Fort Greene|40.69169|-73.97185|Entire home/apt| 215| 2| 198| 1.72| 1| 321|
7322| 18946| Manhattan| Chelsea|40.74192|-73.99501| Private room| 140| 1| 260| 2.12| 1| 12|
7726| 20950| Brooklyn| Crown Heights|40.67592|-73.94694|Entire home/apt| 99| 3| 53| 4.44| 1| 21|
7750| 17985| Manhattan| East Harlem|40.79685|-73.94872|Entire home/apt| 190| 7| 0| 0.0| 2| 249|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
only showing top 20 rows

In [16]:
# lets find the distinct unique valyes in neighbourhood_group
df.select('neighbourhood_group').distinct().show()

+-------------------+
neighbourhood_group|
+-------------------+
 Queens|
 Brooklyn|
 Staten Island|
 Manhattan|
 Bronx|
+-------------------+

In [17]:
# lets find the distinct unique valyes in neighbourhood
df.select('neighbourhood').distinct().show()

+------------------+
 neighbourhood|
+------------------+
 Corona|
 Richmondtown|
 Prince's Bay|
 Westerleigh|
 Mill Basin|
 Civic Center|
 Douglaston|
 Mount Hope|
 Marble Hill|
 Rego Park|
 Dyker Heights|
 Kew Gardens Hills|
 Dongan Hills|
Financial District|
 Bay Terrace|
 Midtown|
 Jackson Heights|
 Bayside|
 Hell's Kitchen|
 Highbridge|
+------------------+
only showing top 20 rows

In [18]:
# lets  find the distinct unique valyes in neighbourhood
df.select('room_type').distinct().show()

+---------------+
 room_type|
+---------------+
 Shared room|
Entire home/apt|
 Private room|
+---------------+

In [19]:
df.createOrReplaceTempView('ab_newyork_view')

In [20]:
spark.sql("select * from ab_newyork_view").show()

+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group| neighbourhood|latitude|longitude| room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
2539| 2787| Brooklyn| Kensington|40.64749|-73.97237| Private room| 149| 1| 9| 0.21| 6| 365|
2595| 2845| Manhattan| Midtown|40.75362|-73.98377|Entire home/apt| 225| 1| 45| 0.38| 2| 355|
3647| 4632| Manhattan| Harlem|40.80902| -73.9419| Private room| 150| 3| 0| 0.0| 1| 365|
3831| 4869| Brooklyn| Clinton Hill|40.68514|-73.95976|Entire home/apt| 89| 1| 270| 4.64| 1| 194|
5022| 7192| Manhattan| East Harlem|40.79851|-73.94399|Entire home/apt| 80| 10| 9| 0.1| 1| 0|
5099| 7322| Manhattan| Murray Hill|40.74767| -73.975|Entire home/apt| 200| 3| 74| 0.59| 1| 129|
5121| 7356| Brooklyn|Bedford-Stuyvesant|40.68688|-73.95596| Private room| 60| 45| 49| 0.4| 1| 0|
5178| 8967| Manhattan| Hell's Kitchen|40.76489|-73.98493| Private room| 79| 2| 430| 3.47| 1| 220|
5203| 7490| Manhattan| Upper West Side|40.80178|-73.96723| Private room| 79| 2| 118| 0.99| 1| 0|
5238| 7549| Manhattan| Chinatown|40.71344|-73.99037|Entire home/apt| 150| 1| 160| 1.33| 4| 188|
5295| 7702| Manhattan| Upper West Side|40.80316|-73.96545|Entire home/apt| 135| 5| 53| 0.43| 1| 6|
5441| 7989| Manhattan| Hell's Kitchen|40.76076|-73.98867| Private room| 85| 2| 188| 1.5| 1| 39|
5803| 9744| Brooklyn| South Slope|40.66829|-73.98779| Private room| 89| 4| 167| 1.34| 3| 314|
6021| 11528| Manhattan| Upper West Side|40.79826|-73.96113| Private room| 85| 2| 113| 0.91| 1| 333|
6090| 11975| Manhattan| West Village| 40.7353|-74.00525|Entire home/apt| 120| 90| 27| 0.22| 1| 0|
6848| 15991| Brooklyn| Williamsburg|40.70837|-73.95352|Entire home/apt| 140| 2| 148| 1.2| 1| 46|
7097| 17571| Brooklyn| Fort Greene|40.69169|-73.97185|Entire home/apt| 215| 2| 198| 1.72| 1| 321|
7322| 18946| Manhattan| Chelsea|40.74192|-73.99501| Private room| 140| 1| 260| 2.12| 1| 12|
7726| 20950| Brooklyn| Crown Heights|40.67592|-73.94694|Entire home/apt| 99| 3| 53| 4.44| 1| 21|
7750| 17985| Manhattan| East Harlem|40.79685|-73.94872|Entire home/apt| 190| 7| 0| 0.0| 2| 249|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
only showing top 20 rows

In [21]:
spark.sql("select distinct(room_type) from ab_newyork_view").show()

+---------------+
 room_type|
+---------------+
 Shared room|
Entire home/apt|
 Private room|
+---------------+

In [22]:
spark.sql("select count(id) from ab_newyork_view where room_type='-73.90783'").show()

+---------+
count(id)|
+---------+
 0|
+---------+

In [23]:
spark.sql("select count(id) as number_of_bookings,neighbourhood_group from ab_newyork_view group by neighbourhood_group order by number_of_bookings desc").show()

+------------------+-------------------+
number_of_bookings|neighbourhood_group|
+------------------+-------------------+
 21595| Manhattan|
 20056| Brooklyn|
 5627| Queens|
 1080| Bronx|
 369| Staten Island|
+------------------+-------------------+

In [24]:
spark.sql("select count(id) as number_of_bookings,neighbourhood from ab_newyork_view group by neighbourhood order by number_of_bookings desc").show()

+------------------+------------------+
number_of_bookings| neighbourhood|
+------------------+------------------+
 3914| Williamsburg|
 3705|Bedford-Stuyvesant|
 2645| Harlem|
 2461| Bushwick|
 1968| Upper West Side|
 1950| Hell's Kitchen|
 1849| East Village|
 1790| Upper East Side|
 1560| Crown Heights|
 1541| Midtown|
 1114| Greenpoint|
 1112| East Harlem|
 1111| Chelsea|
 911| Lower East Side|
 896| Astoria|
 895|Washington Heights|
 767| West Village|
 744|Financial District|
 619| Flatbush|
 571| Clinton Hill|
+------------------+------------------+
only showing top 20 rows

In [25]:
spark.sql("select count(id) as number_of_bookings,room_type from ab_newyork_view group by room_type order by number_of_bookings desc").show()

+------------------+---------------+
number_of_bookings| room_type|
+------------------+---------------+
 25346|Entire home/apt|
 22225| Private room|
 1156| Shared room|
+------------------+---------------+

In [26]:
spark.sql("select count(neighbourhood) as number_of_365_neighbourhoods,neighbourhood_group from ab_newyork_view where availability_365 = 365 group by neighbourhood_group\
          order by number_of_365_neighbourhoods desc").show()

+----------------------------+-------------------+
number_of_365_neighbourhoods|neighbourhood_group|
+----------------------------+-------------------+
 568| Manhattan|
 452| Brooklyn|
 200| Queens|
 54| Bronx|
 11| Staten Island|
+----------------------------+-------------------+

In [27]:
spark.sql("select sum(number_of_reviews) as Reviews_Count,neighbourhood,neighbourhood_group,room_type,price from ab_newyork_view group by\
          neighbourhood,neighbourhood_group,room_type,price order by Reviews_Count desc limit 10").show()

+-------------+------------------+-------------------+---------------+-----+
Reviews_Count| neighbourhood|neighbourhood_group| room_type|price|
+-------------+------------------+-------------------+---------------+-----+
 3787|Bedford-Stuyvesant| Brooklyn| Private room| 65|
 3640|Bedford-Stuyvesant| Brooklyn|Entire home/apt| 125|
 3625| Harlem| Manhattan| Private room| 60|
 3500|Bedford-Stuyvesant| Brooklyn|Entire home/apt| 100|
 3176|Bedford-Stuyvesant| Brooklyn|Entire home/apt| 120|
 3149|Bedford-Stuyvesant| Brooklyn| Private room| 60|
 3134|Bedford-Stuyvesant| Brooklyn|Entire home/apt| 150|
 3001|Bedford-Stuyvesant| Brooklyn| Private room| 50|
 2999| Harlem| Manhattan| Private room| 65|
 2850|Bedford-Stuyvesant| Brooklyn|Entire home/apt| 135|
+-------------+------------------+-------------------+---------------+-----+

###Linear Regression

In [29]:
# Import the required libraries

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [30]:
# Use StringIndexer to convert the categorical columns to hold numerical data

neighbourhood_group_indexer = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index',handleInvalid='keep')
room_type_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')

In [31]:
assembler = VectorAssembler(inputCols=['neighbourhood_group_index','latitude','longitude',
                                       'room_type_index','minimum_nights','number_of_reviews',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [32]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[neighbourhood_group_indexer,room_type_indexer,assembler])

In [33]:
# Create a 70-30 train test split

train_data,test_data=df.randomSplit([0.7,0.3])

In [34]:
fit_pipe=pipe.fit(train_data)

In [35]:
train_data=fit_pipe.transform(train_data)

In [36]:
lr_model = LinearRegression(labelCol='price')

In [37]:
fit_model = lr_model.fit(train_data.select(['features','price']))

In [38]:
test_data=fit_pipe.transform(test_data)

In [39]:
lr_results = fit_model.transform(test_data)

In [40]:
lr_results.select(['price','prediction']).show()

+-----+------------------+
price| prediction|
+-----+------------------+
 89|134.45771015030914|
 80|204.06032561839675|
 60| 57.55053925798711|
 150|208.25052547585074|
 135| 200.6532781079295|
 85| 67.82507800916937|
 80|153.75986932910746|
 110|129.96898241668532|
 80|104.55903410219616|
 150|219.13493226178252|
 50|109.70043883055769|
 89|108.93721706075303|
 150| 157.6725703395059|
 135|153.98429678566026|
 110|217.11349972715107|
 80| 83.19579006252752|
 110|179.46960401700198|
 69| 92.39710281808948|
 180|248.56230584460718|
 80|126.98405752473627|
+-----+------------------+
only showing top 20 rows

In [41]:
test_results = fit_model.evaluate(test_data)

In [42]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -45.45771015030914|
-124.06032561839675|
 2.4494607420128887|
-58.250525475850736|
 -65.65327810792951|
 17.174921990830626|
 -73.75986932910746|
 -19.96898241668532|
-24.559034102196165|
 -69.13493226178252|
 -59.70043883055769|
-19.937217060753028|
 -7.67257033950591|
-18.984296785660263|
-107.11349972715107|
-3.1957900625275215|
 -69.46960401700198|
 -23.39710281808948|
 -68.56230584460718|
 -46.98405752473627|
+-------------------+
only showing top 20 rows

In [43]:
test_results.rootMeanSquaredError

Out[33]: 239.20805621732865

In [44]:
test_results.r2

Out[34]: 0.07823480715786313

In [45]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
pd_df2 = lr_results.toPandas()
print(pd_df2)

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
 id ... prediction
0 3831 ... 134.457710
1 5022 ... 204.060326
2 5121 ... 57.550539
3 5238 ... 208.250525
4 5295 ... 200.653278
5 5441 ... 67.825078
6 8025 ... 153.759869
7 8110 ... 129.968982
8 8700 ... 104.559034
9 9357 ... 219.134932
10 9668 ... 109.700439
11 10962 ... 108.937217
12 11943 ... 157.672570
13 12318 ... 153.984297
14 12940 ... 217.113500
15 13394 ... 83.195790
16 14991 ... 179.469604
17 15220 ... 92.397103
18 15341 ... 248.562306
19 15385 ... 126.984058
20 16421 ... 99.323036
21 16821 ... 156.245205
22 17693 ... 166.234348
23 18127 ... 262.773336
24 18198 ... 98.852205
25 18590 ... 66.996907
26 19282 ... 182.152246
27 19601 ... 188.855086
28 20724 ... 133.846943
29 20734 ... 176.731603
... ... ... ...
14765 36448708 ... 148.747618
14766 36449668 ... 99.420359
14767 36449743 ... 209.140170
14768 36449938 ... 231.556852
14769 36449941 ... 99.322159
14770 36450435 ... 195.490378
14771 36450814 ... 55.273721
14772 36450896 ... 241.742426
14773 36451212 ... 130.133517
14774 36451362 ... 82.733255
14775 36452965 ... 170.185286
14776 36453030 ... 151.825291
14777 36453642 ... 144.164790
14778 36453952 ... 310.743895
14779 36454025 ... 145.658972
14780 36455321 ... 86.566959
14781 36455402 ... 213.636497
14782 36455579 ... 219.901407
14783 36455649 ... 91.500162
14784 36456548 ... 137.760915
14785 36456829 ... 22.410479
14786 36468375 ... 156.284124
14787 36468409 ... 211.053067
14788 36468880 ... 77.434826
14789 36469741 ... 56.831922
14790 36474911 ... 183.391060
14791 36475746 ... 113.806338
14792 36476675 ... 226.848932
14793 36485057 ... 76.942108
14794 36485609 ... 20.324010

[14795 rows x 17 columns]

In [47]:
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.scatterplot(pd_df2['prediction'],pd_df2['price'])
display(plt.show())

d ###Decision Tree

In [49]:
# Import the required libraries

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [50]:
neighbourhood_group_indexer = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index',handleInvalid='keep')
room_type_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')

In [51]:
assembler = VectorAssembler(inputCols=['neighbourhood_group_index','latitude','longitude',
                                       'room_type_index','minimum_nights','number_of_reviews',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [52]:
train,test=df.randomSplit([0.7,0.3])

In [53]:
dt_model=DecisionTreeRegressor(featuresCol="features",labelCol='price')

In [54]:
pipe = Pipeline(stages=[neighbourhood_group_indexer,room_type_indexer,assembler,dt_model])

In [55]:
train_model = pipe.fit(train)

In [56]:
test_result = train_model.transform(test)

In [57]:
test_result.select(['price','prediction']).show()

+-----+------------------+
price| prediction|
+-----+------------------+
 225| 716.8850574712644|
 150|241.15727002967358|
 135|241.15727002967358|
 85|126.48326100433974|
 89| 71.25784313725491|
 85| 90.00184049079755|
 140| 189.1784302653868|
 299| 189.1784302653868|
 130| 71.25784313725491|
 44| 66.49546436285097|
 50| 90.00184049079755|
 89| 71.25784313725491|
 68|126.48326100433974|
 150| 145.3427922814983|
 151|241.15727002967358|
 80| 90.00184049079755|
 375|241.15727002967358|
 250| 189.1784302653868|
 55|126.48326100433974|
 52|126.48326100433974|
+-----+------------------+
only showing top 20 rows

In [58]:
eval = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")

In [59]:
rmse = eval.evaluate(test_result)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 229.387

In [60]:
eval1 = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="r2")

In [61]:
r2 = eval1.evaluate(test_result)
print("R Squared (R2) on test data = %g" % r2)

R Squared (R2) on test data = 0.129301

In [62]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
pd_df3 = test_result.toPandas()
print(pd_df3)

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
 id ... prediction
0 2595 ... 716.885057
1 5238 ... 241.157270
2 5295 ... 241.157270
3 5441 ... 126.483261
4 5803 ... 71.257843
5 6021 ... 90.001840
6 6848 ... 189.178430
7 7801 ... 189.178430
8 8024 ... 71.257843
9 9518 ... 66.495464
10 9668 ... 90.001840
11 10962 ... 71.257843
12 12192 ... 126.483261
13 12627 ... 145.342792
14 14287 ... 241.157270
15 15385 ... 90.001840
16 15396 ... 241.157270
17 15711 ... 189.178430
18 16338 ... 126.483261
19 16421 ... 126.483261
20 16974 ... 189.178430
21 18127 ... 241.157270
22 19159 ... 189.178430
23 19169 ... 241.157270
24 19812 ... 189.178430
25 20300 ... 126.483261
26 20853 ... 171.768915
27 21456 ... 189.178430
28 21644 ... 90.001840
29 22911 ... 135.511795
... ... ... ...
14612 36426720 ... 126.483261
14613 36441134 ... 276.113355
14614 36442487 ... 135.511795
14615 36445224 ... 86.594883
14616 36446437 ... 126.483261
14617 36447683 ... 189.178430
14618 36447795 ... 189.178430
14619 36449938 ... 189.178430
14620 36450283 ... 189.178430
14621 36450814 ... 69.579759
14622 36451138 ... 86.594883
14623 36452721 ... 69.579759
14624 36453952 ... 716.885057
14625 36455402 ... 145.342792
14626 36455579 ... 189.178430
14627 36455649 ... 86.594883
14628 36455809 ... 66.495464
14629 36456829 ... 90.001840
14630 36457832 ... 175.027607
14631 36468386 ... 189.178430
14632 36468409 ... 189.178430
14633 36471896 ... 90.001840
14634 36476675 ... 276.113355
14635 36479230 ... 61.092421
14636 36481615 ... 86.594883
14637 36482783 ... 90.001840
14638 36482809 ... 86.594883
14639 36484665 ... 69.579759
14640 36485057 ... 86.594883
14641 36485431 ... 189.178430

[14642 rows x 17 columns]

In [64]:
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.scatterplot(pd_df3['prediction'],pd_df3['price'])
display(plt.show())

###Random Forest

In [66]:
# Import the required libraries

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [67]:
neighbourhood_group_indexer = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index',handleInvalid='keep')
room_type_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')

In [68]:
assembler = VectorAssembler(inputCols=['neighbourhood_group_index','latitude','longitude',
                                       'room_type_index','minimum_nights','number_of_reviews',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [69]:
train_dt,test_dt=df.randomSplit([0.7,0.3])

In [70]:
rf_model=RandomForestRegressor(featuresCol="features",labelCol='price')

In [71]:
pipe = Pipeline(stages=[neighbourhood_group_indexer,room_type_indexer,assembler,rf_model])

In [72]:
model = pipe.fit(train_dt)

In [73]:
result = model.transform(test_dt)

In [74]:
result.select(['price','prediction']).show()

+-----+------------------+
price| prediction|
+-----+------------------+
 225|305.91414834802873|
 89|167.76670265993627|
 200|257.13750933397705|
 60| 91.2552153604507|
 79|157.00545669212948|
 135| 212.8529918421385|
 85| 91.95620847062312|
 140| 155.3457129158882|
 215| 201.6545474252552|
 299|182.54137922954547|
 80| 97.67319255855219|
 110| 93.14214348403164|
 52| 91.45374716138356|
 35|117.20607831551327|
 150|133.79465989262093|
 120|100.25873121567821|
 150| 166.6997728344316|
 115|151.76433079007563|
 151| 265.1677956392732|
 228|143.90208500526305|
+-----+------------------+
only showing top 20 rows

In [75]:
evaluate = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")

In [76]:
rmse = evaluate.evaluate(result)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 177.66

In [77]:
evaluate1 = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="r2")

In [78]:
rsq = evaluate1.evaluate(test_result)
print("R Squared (R2) on test data = %g" % rsq)

R Squared (R2) on test data = 0.129301

In [79]:
pd_df4 = result.toPandas()
print(pd_df4)

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
 id ... prediction
0 2595 ... 305.914148
1 3831 ... 167.766703
2 5099 ... 257.137509
3 5121 ... 91.255215
4 5178 ... 157.005457
5 5295 ... 212.852992
6 6021 ... 91.956208
7 6848 ... 155.345713
8 7097 ... 201.654547
9 7801 ... 182.541379
10 8025 ... 97.673193
11 8110 ... 93.142143
12 9704 ... 91.453747
13 11452 ... 117.206078
14 11943 ... 133.794660
15 12303 ... 100.258731
16 12627 ... 166.699773
17 13050 ... 151.764331
18 14287 ... 265.167796
19 14290 ... 143.902085
20 15341 ... 261.839807
21 16338 ... 100.517513
22 16421 ... 157.005457
23 16580 ... 133.455000
24 16821 ... 150.221146
25 17037 ... 253.119410
26 18152 ... 102.894024
27 19159 ... 181.071557
28 19169 ... 263.237224
29 19282 ... 152.741891
... ... ... ...
14575 36441908 ... 145.621789
14576 36442252 ... 135.359166
14577 36442487 ... 164.006702
14578 36444527 ... 82.845600
14579 36445121 ... 97.427041
14580 36446225 ... 170.430219
14581 36446514 ... 72.330510
14582 36448400 ... 148.280417
14583 36449941 ... 72.119377
14584 36450318 ... 299.598718
14585 36451138 ... 74.550216
14586 36451212 ... 90.502485
14587 36452965 ... 167.909371
14588 36453160 ... 341.913882
14589 36453642 ... 71.239885
14590 36453952 ... 682.323662
14591 36454025 ... 86.192079
14592 36454717 ... 72.518497
14593 36455402 ... 170.839461
14594 36455579 ... 184.924094
14595 36457832 ... 186.970808
14596 36458668 ... 92.098822
14597 36468409 ... 208.385648
14598 36469741 ... 80.444202
14599 36472710 ... 164.389677
14600 36474023 ... 80.439281
14601 36476675 ... 265.378085
14602 36479723 ... 69.683444
14603 36482783 ... 108.039968
14604 36487245 ... 162.658369

[14605 rows x 17 columns]

In [80]:
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.scatterplot(pd_df4['prediction'],pd_df4['price'])
display(plt.show())